<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: 
* Reward function: 
* State: 
* Environment: 

<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages

In [ ]:
!pip install -r requirements.txt --user

<a id='1.2'></a>
## 2.2. Import Packages

In [ ]:
%load_ext autoreload
%autoreload 
import pandas as pd
from config import config
from dataset.download_dataset.cryptodownloader_binance import CryptoDownloader_binance
from preprocessing.preprocessors import FeatureEngineer
from preprocessing.data import data_split
from env.env_custom import CustomTradingEnv
from model.models import DRLAgent
from trade.backtest import BackTest
import warnings
warnings.filterwarnings("ignore")



<a id='1.3'></a>
## 2.3 Create Folders

In [ ]:
import os
download_data = False
if not os.path.exists(config.DATA_SAVE_DIR):
    os.makedirs(config.DATA_SAVE_DIR)
    download_data = True
if not os.path.exists(config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists(config.TENSORBOARD_LOG_DIR):
    os.makedirs(config.TENSORBOARD_LOG_DIR)
if not os.path.exists(config.RESULTS_DIR):
    os.makedirs(config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data

In [ ]:
data_downloader = CryptoDownloader_binance(config.START_DATE, config.END_DATE, config.MULTIPLE_TICKER_8, config.DATA_SAVE_DIR, config.DATA_GRANULARITY)
if download_data:    
    data_downloader.download_data()
df = data_downloader.load()

In [ ]:
df

# Part 4: Preprocess Data

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = True,
                    use_covariance= True
)

df = fe.preprocess_data(df)

In [ ]:
df.head()

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2020-02-02 to 2020-12-31

In [ ]:
train = data_split(df, '2020-02-02','2020-12-31')

## Environment for Portfolio Allocation


In [ ]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
features =  config.TECHNICAL_INDICATORS_SHORTPERIOD + config.TECHNICAL_INDICATORS_LONGPERIOD  + ["open", "close", 'high', 'low']
features += [f"{feature}_diff" for feature in features]
features += [feature for feature in df.columns if feature.startswith("cov_")]

env_kwargs = {
    "initial_amount": 1000000, 
    "technical_indicator_list": features, 
    "max_assets_amount_per_trade": 100, 
    "main_tickers": train.tic.unique(),
    "all_tickers": train.tic.unique(),
    "reward_scaling": 1e-4,
    "comission_value": 0.01
    
}

e_train_gym = CustomTradingEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()

<a id='5'></a>
# Part 6: Implement DRL Algorithms

In [ ]:
# initialize
agent = DRLAgent(env = env_train)

### Model 1: **A2C**


In [ ]:
A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS, tensorboard_log = config.TENSORBOARD_LOG_DIR)

In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=100)

### Model 2: **PPO**

In [ ]:
# agent = DRLAgent(env = env_train)
# PPO_PARAMS = {
#     "n_steps": 2048,
#     "ent_coef": 0.005,
#     "learning_rate": 0.0001,
#     "batch_size": 128,
# }
# model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
# trained_ppo = agent.train_model(model=model_ppo, 
#                              tb_log_name='ppo',
#                              total_timesteps=80000)

## Trading
Assume that we have $1,000,000 initial capital at 2021-01-01.

In [ ]:
trade = data_split(df,config.START_TEST_DATE, config.END_DATE)
e_trade_gym = CustomTradingEnv(df = trade, **env_kwargs)

In [ ]:
trade.shape

In [ ]:
trade

In [ ]:
allocations, transactions, allocation_values = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

In [ ]:
allocations

In [ ]:
transactions

In [ ]:
allocation_values

<a id='6'></a>
# Part 7: Backtest

In [ ]:
%load_ext autoreload
%autoreload 
bat = BackTest(trained_a2c, e_trade_gym)
bat.create_summary(allocations, allocation_values, transactions)

In [ ]:
bat.plot_return_against_hold(allocation_values)

In [ ]:
%load_ext autoreload
%autoreload 
from trade.time_series_validation import TimeSeriesValidation
tsv = TimeSeriesValidation(num_splits=5, total_timesteps_model=1000, with_graphs=False)

model_name = "a2c"
model_params = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}

stock_dimension = len(df.tic.unique())
state_space = stock_dimension
features =  config.TECHNICAL_INDICATORS_SHORTPERIOD + config.TECHNICAL_INDICATORS_LONGPERIOD + ["open", "close", 'high', 'low']
features += [f"{feature}_diff" for feature in features]
features += [feature for feature in df.columns if feature.startswith("cov_")]

env_params = {
    "initial_amount": 1000000, 
    "technical_indicator_list": features, 
    "max_assets_amount_per_trade": 100, 
    "main_tickers": df.tic.unique(),
    "all_tickers": df.tic.unique(),
    "reward_scaling": 1e-4,
    "comission_value": 0.01
    
}


results = tsv.run(df, env_params, model_name, model_params)

In [ ]:
results